# Exercise 10

## KNN exercise with NBA player data

## Introduction

- NBA player statistics from 2014-2015 (partial season): [data](https://github.com/justmarkham/DAT4-students/blob/master/kerry/Final/NBA_players_2015.csv), [data dictionary](https://github.com/justmarkham/DAT-project-examples/blob/master/pdf/nba_paper.pdf)
- **Goal:** Predict player position using assists, steals, blocks, turnovers, and personal fouls

## Read the data into Pandas

In [1]:
# read the data into a DataFrame
import pandas as pd
url = 'https://raw.githubusercontent.com/justmarkham/DAT4-students/master/kerry/Final/NBA_players_2015.csv'
nba = pd.read_csv(url, index_col=0)

In [2]:
# examine the columns
nba.columns

Index(['season_end', 'player', 'pos', 'age', 'bref_team_id', 'g', 'gs', 'mp',
       'fg', 'fga', 'fg_', 'x3p', 'x3pa', 'x3p_', 'x2p', 'x2pa', 'x2p_', 'ft',
       'fta', 'ft_', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf',
       'pts', 'G', 'MP', 'PER', 'TS%', '3PAr', 'FTr', 'TRB%', 'AST%', 'STL%',
       'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM',
       'BPM', 'VORP'],
      dtype='object')

In [3]:
# examine the positions
nba.pos.value_counts()

G    200
F    199
C     79
Name: pos, dtype: int64

## Create X and y

Use the following features: assists, steals, blocks, turnovers, personal fouls

In [4]:
# map positions to numbers
nba['pos_num'] = nba.pos.map({'C':0, 'F':1, 'G':2})

In [5]:
# create feature matrix (X)
feature_cols = ['ast', 'stl', 'blk', 'tov', 'pf']
X = nba[feature_cols]

In [6]:
# alternative way to create X
X = nba.loc[:, 'ast':'pf']

In [7]:
# create response vector (y)
y = nba.pos_num

# Exercice 10.1

* Split the data in train and test
* Train a KNN model (K=5)
* Evaluate the accuracy

In [20]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, random_state=123)

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [21]:
y_pred = knn.predict(X_test)
y_pred_prob = knn.predict_proba(X_test)
(y_pred==y_test).mean()

0.69999999999999996

# Exercice 10.2 

Predict player position and calculate predicted probability of each position

Predict for a player with these statistics: 1 assist, 1 steal, 0 blocks, 1 turnover, 2 personal fouls

In [13]:
# create a list to represent a player
import numpy as np
player = np.array([1, 1, 0, 1, 2]).reshape(1, -1) 

y_pred = knn.predict(player)
y_pred


array([2], dtype=int64)

In [14]:
y_pred_prob = knn.predict_proba(player)
y_pred_prob[:5]

array([[ 0. ,  0.4,  0.6]])

# Exercice 10.3  

Repeat steps 10.1 and 10.2 using K=50

In [16]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, random_state=123)

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=50)
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=50, p=2,
           weights='uniform')

In [17]:
(y_pred==y_test).mean()


0.36666666666666664

In [18]:
# create a list to represent a player
import numpy as np
player = np.array([1, 1, 0, 1, 2]).reshape(1, -1) 

y_pred = knn.predict(player)
y_pred

array([1], dtype=int64)

In [19]:
y_pred_prob = knn.predict_proba(player)
y_pred_prob[:5]

array([[ 0.06,  0.52,  0.42]])

# Exercice 10.4 (3 points) 

Explore the features to decide which ones are predictive

In [26]:
feature_selection=['season_end', 'player', 'pos', 'age', 'bref_team_id', 'g', 'gs', 'mp',
       'fg', 'fga', 'fg_', 'x3p', 'x3pa', 'x3p_', 'x2p', 'x2pa', 'x2p_', 'ft',
       'fta', 'ft_', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf',
       'pts', 'G', 'MP', 'PER', 'TS%', '3PAr', 'FTr', 'TRB%', 'AST%', 'STL%',
       'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM',
       'BPM', 'VORP']
X=nba[list(feature_selection)]

from sklearn.feature_selection import SelectKBest
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
t=[]



for i in range(1,20):
    sel = SelectKBest(k=i)
    sel.fit(X, y)
    sel.get_support()
    X_sel = sel.transform(X)
    results.loc['k='+str(i)] = pd.Series(cross_val_score(logreg, X_sel, y, cv=10, scoring='accuracy')).mean()

results

ValueError: could not convert string to float: 'BOS'